In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from library import datasets, models
from flax import linen as nn
from jax import random, numpy as jnp
import optax
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'notebook_connected'

In [3]:
LATENT_DIM = 1
AMBIENT_DIM = 2

In [4]:
taylor_pair_dis = datasets.point_dataset.PointPairDistribution.random_taylor(
    latent_dim=LATENT_DIM,
    dis_A_dim=AMBIENT_DIM,
    dis_B_dim=AMBIENT_DIM,
    latent_range=1,
    max_order=2,
    coeff_range=1,
    noise_std_A=0.05,
    noise_std_B=0,
    key=random.PRNGKey(2))

dataset = taylor_pair_dis.generate_dataset(1000)

In [5]:
A, _ = dataset.get_all_point_pairs()

In [6]:
discriminator = nn.Sequential([
    nn.Dense(8),
    nn.relu,
    nn.Dense(4),
    nn.relu,
    nn.Dense(1),
])

generator = nn.Sequential([
    nn.Dense(8),
    nn.relu,
    nn.Dense(8),
    nn.relu,
    nn.Dense(4),
    nn.relu,
    nn.Dense(AMBIENT_DIM),
])

model = models.vanilla_gan.VanillaGAN(
    generator=generator,
    discriminator=discriminator,
    latent_shape=(LATENT_DIM,),
    ambient_shape=(AMBIENT_DIM,)
)

print(model)
model.initialize(loss_fn=optax.sigmoid_binary_cross_entropy)

generator:
	Model Structure:
		Sequential(
		    # attributes
		    layers = [Dense(
		        # attributes
		        features = 8
		        use_bias = True
		        dtype = None
		        param_dtype = float32
		        precision = None
		        kernel_init = init
		        bias_init = zeros
		    ), <jax._src.custom_derivatives.custom_jvp object at 0x7fbd7eac5d20>, Dense(
		        # attributes
		        features = 8
		        use_bias = True
		        dtype = None
		        param_dtype = float32
		        precision = None
		        kernel_init = init
		        bias_init = zeros
		    ), <jax._src.custom_derivatives.custom_jvp object at 0x7fbd7eac5d20>, Dense(
		        # attributes
		        features = 4
		        use_bias = True
		        dtype = None
		        param_dtype = float32
		        precision = None
		        kernel_init = init
		        bias_init = zeros
		    ), <jax._src.custom_derivatives.custom_jvp object at 0x7fbd7eac5d20>, Dense(
		        # attributes
		        

In [34]:
model.train(A,optax.sgd(learning_rate=1e-3), print_every=5, batch_size=64, num_epochs=250)

iteration 10; gen_loss: 0.007669384125620127; dis_loss: 0.691248893737793; gen_grads_magnitude: 0.0014262356433622485; dis_grads_magnitude: 0.03456992739123483: : 15it [00:00, 38.34it/s]
iteration 10; gen_loss: 0.014459680765867233; dis_loss: 0.6898103356361389; gen_grads_magnitude: 0.0023365823576233053; dis_grads_magnitude: 0.03836700307038181: : 15it [00:00, 89.48it/s]
iteration 10; gen_loss: 0.017902921885252; dis_loss: 0.6894446611404419; gen_grads_magnitude: 0.002736587020842923; dis_grads_magnitude: 0.03135024138810477: : 15it [00:00, 94.74it/s]
iteration 10; gen_loss: 0.027687206864356995; dis_loss: 0.6874611377716064; gen_grads_magnitude: 0.004099892804342541; dis_grads_magnitude: 0.03756775069183685: : 15it [00:00, 92.11it/s]
iteration 10; gen_loss: 0.02872132882475853; dis_loss: 0.6878052353858948; gen_grads_magnitude: 0.004174335867159502; dis_grads_magnitude: 0.030025445101820133: : 15it [00:00, 93.75it/s]
iteration 10; gen_loss: 0.0417398139834404; dis_loss: 0.68484055995

In [35]:
gan_distribution = model.create_distribution()

In [36]:
true_samples = A

In [37]:
fake_samples = gan_distribution.draw_samples(random.normal(random.PRNGKey(0), (1000, LATENT_DIM)))

In [38]:
print(true_samples.shape, fake_samples.shape)

(1000, 2) (1000, 2)


In [39]:
df = {'x': jnp.concatenate((true_samples[:, 0], fake_samples[:, 0]), axis=0),
      'y': jnp.concatenate((true_samples[:, 1], fake_samples[:, 1]), axis=0),
      'labels': jnp.concatenate((jnp.ones((true_samples.shape[0],)), jnp.zeros((fake_samples.shape[0],))), axis=0)}

In [40]:
px.scatter(df, x='x', y='y', color='labels')